# Alfano et analysis

In [2]:
# libraries 
# libraries
suppressMessages(library(dplyr))
suppressMessages(library(Seurat))
suppressMessages(library(Matrix))
suppressMessages(library(gplots))
suppressMessages(library(ggplot2))
suppressMessages(library(openxlsx))
suppressMessages(library(cowplot))
suppressMessages(library(patchwork))
suppressMessages(library(repr))
suppressMessages(library(cowplot))
suppressMessages(library("viridis"))
suppressMessages(library('pals'))
suppressMessages(library(wesanderson))


In [ ]:
setwd("/Users/tascini.annasofia/ctgb_cluster_root/lustre2/scratch/bioinfotree/common/bioinfotree/prj/AlfanoM_904_infertilita_epigenetics/dataset/20200110/seraut")
Npz='OA'
#Npz='iNOA1'
#Npz='iNOA2'
#Npz='iNOA3'
dir.create(paste('./NEW_pz_',Npz, sep=''), showWarnings = F, recursive = T)

In [ ]:
# Load data generated with UMI
# USE this for iNOA1 iNOA2 iNOA3 OA
counts_file=paste('samples/', Npz,
                  '/runs/all/fastq/merged/umi-extracted/mapped/STAR/featureCounts/sorted/samtools/umi-counts/',
                    Npz,'.counts.matrix.tsv.gz',sep='')
counts_file
UMImatrix_sparse = Matrix(as.matrix(read.table(counts_file, 
                                               sep="\t", 
                                               header = T, 
                                               quote = "", 
                                               row.names = 1)), 
                          sparse=TRUE)

In [ ]:
# USE this for Guo et al
# import data from GSE112013
counts_file=('/Users/tascini.annasofia/Downloads/GSE112013_Combined_UMI_table.txt')
UMImatrix_sparse = Matrix(as.matrix(read.table(counts_file, 
                                               sep="\t", 
                                               header = T, 
                                               quote = "", 
                                               row.names = 1)), 
                          sparse=TRUE)

In [ ]:
# Initialize the Seurat object with the raw (non-normalized data).
pz <- CreateSeuratObject(counts = UMImatrix_sparse, 
                         project = paste("pz_",Npz,sep=''), 
                         min.cells = 5, 
                         min.features = 50)

In [ ]:
#pz.iNOA1.raw <- pz
#pz.iNOA2.raw <- pz
#pz.iNOA3.raw <- pz
#pz.GSE112013.raw <- pz
pz.OA.raw <- pz

In [ ]:
pz[["percent.mt"]] <- PercentageFeatureSet(pz, pattern = "^MT-")
## QC plots
options(repr.plot.width=8, repr.plot.height=6)
VlnPlot(pz, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
plot1 <- FeatureScatter(pz, feature1 = "nCount_RNA", feature2 = "percent.mt",cols = NULL)
plot2 <- FeatureScatter(pz, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
CombinePlots(plots = list(plot1, plot2))

In [ ]:
sum(pz@meta.data$percent.mt < 20)
FeatureScatter(pz, feature1 = "nCount_RNA", feature2 = "percent.mt",cols = NULL)

In [ ]:
if (Npz == 'iNOA1') {
  min_nFeature_RNA = 500
  max_nFeature_RNA = 6000
  max_percent_MT = 20
}


if (Npz == 'iNOA2') {
  min_nFeature_RNA = 500
  max_nFeature_RNA = 6000
  max_percent_MT = 20
}

if (Npz == 'iNOA3') {
  min_nFeature_RNA = 500
  max_nFeature_RNA = 6000
  max_percent_MT = 20
}

if (Npz == 'OA') {
  min_nFeature_RNA = 150
  max_nFeature_RNA = 6000
  max_percent_MT = 20
}

if (Npz == 'GSE112013') {
  min_nFeature_RNA = 500
  max_nFeature_RNA = 10000
  max_percent_MT = 20
}

pz <- subset(pz, 
             subset = nFeature_RNA > min_nFeature_RNA & nFeature_RNA < max_nFeature_RNA & percent.mt < max_percent_MT)
VlnPlot(pz, 
        features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), 
        ncol = 3)
FeatureScatter(pz, feature1 = "nCount_RNA", feature2 = "percent.mt",cols = NULL)

# Normalization
By default, Seurat employs a global-scaling normalization method “LogNormalize”. 
It normalizes the feature expression measurements for each cell by the total expression, 
multiplies this by a scale factor (10,000 by default), and log-transforms the result.
Normalized values are stored in pz[["RNA"]]@data.

In [ ]:
pz <- NormalizeData(pz, 
                    normalization.method = "LogNormalize", 
                    scale.factor = 10000)

# Highly Variable Feature
calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others) focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets. Seraut3 use a different method to Seraut 2

In [ ]:
pz <- FindVariableFeatures(pz, 
                           selection.method = "vst", 
                           nfeatures = 2000)
top20_2 <- head(VariableFeatures(pz), 20) # Identify the 20 most highly variableGenes
plot1 <- VariableFeaturePlot(pz)
LabelPoints(plot = plot1, points = top20_2, repel = TRUE)


# Scaling Data
Apply a linear transformation (‘scaling’) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. 
It is used the ScaleData function: shifts the expression of each gene, so that the mean expression across cells is 0, scales the expression of each gene, so that the variance across cells is 1. This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate. The results of this are stored in pz[["RNA"]]@scale.data. We also regress out for the %MT abd the nUMI

In [ ]:
# scale data and regress out %MT and nUMI
all.genes <- rownames(pz)

#pz <- ScaleData(pz, features = all.genes)

pz <- ScaleData(pz, 
                vars.to.regress = c("percent.mt", "nFeature_RNA"), 
                features = all.genes)

# Linear dimensional reduction 
PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
pz <- RunPCA(pz, 
             features = VariableFeatures(object = pz))
# Examine and visualize PCA results a few different ways
print(pz[["pca"]], dims = 1:5, nfeatures = 5)
VizDimLoadings(pz, dims = 1:3, reduction = "pca")
DimPlot(pz, 
        reduction = "pca")


DimHeatmap allows for easy exploration of the primary sources of heterogeneity in a dataset, 
can be useful when trying to decide which PCs to include for further downstream analyses. Both cells and features are ordered according to their PCA scores. Setting cells to a number plots the ‘extreme’ cells on both ends of the spectrum, which dramatically speeds plotting for large datasets. 
Though clearly a supervised analysis, we find this to be a valuable tool for exploring correlated feature sets.

In [ ]:
DimHeatmap(pz, dims = 1, cells = 500, balanced = TRUE)
DimHeatmap(pz, dims = 1:15, cells = 500, balanced = TRUE)

# CEll Cycle effect

In [ ]:
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
pz <- CellCycleScoring(pz, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)

head(pz[[]], 10)
RidgePlot(pz , features = c("PCNA", "CDC45", "ECT2", "FHL2"), ncol = 2)
pz <- RunPCA(pz, features = VariableFeatures(pz))
DimPlot(pz)

Regress out the cell cycle:

In [ ]:
pz <- ScaleData(pz, 
                vars.to.regress = c("S.Score", "G2M.Score", "percent.mt", "nFeature_RNA"), 
                features = rownames(pz)) 


In [ ]:
pz <- RunPCA(pz, features = VariableFeatures(pz))

DimPlot(pz)

# Determine dimensionality of the dataset 
To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a ‘metafeature’, that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. 
However, how many componenets should we choose to include? 10? 20? 100?
NOTE: This process can take a long time for big datasets, comment out for expediency. More approximate techniques such as those implemented in ElbowPlot() can be used to reduce computation time. 

In [ ]:
## Elbow plot
ElbowPlot(pz, ndims = 50)

In [ ]:
if (Npz == 'iNOA1') {
  nPC = 20 
  res = 0.5
}


if (Npz == 'iNOA2') {
  nPC = 20 
  res = 0.5
}

if (Npz == 'iNOA3') {
  nPC = 20 
  res = 0.2
}

if (Npz == 'OA') {
  nPC = 20 
  res = 0.7
}

In [ ]:
pz <- FindNeighbors(pz, dims = 1:nPC)
pz <- FindClusters(pz, resolution = res)
pz <- RunUMAP(pz, dims = 1:nPC)
pz <- RunTSNE(pz, dims = 1:nPC)
DimPlot(pz, reduction = "umap", label = T) + 
  theme(plot.title = element_text(color="blue", size=26, face="bold.italic"),
        axis.text.x = element_text(angle = 90, face = "bold", color = 'dodgerblue4', size=22, hjust =1), 
        axis.title.x = element_text(face = "bold", color = "dodgerblue2", size = 24),
        axis.text.y = element_text(angle = 0, face = "bold", color = 'dodgerblue4', size=22),
        axis.title.y = element_text(face = "bold", color = "dodgerblue2", size = 24),
        legend.text = element_text(face = "bold", color = "dodgerblue2", size = 22),
        panel.background = element_rect(fill = "white",colour = "black", size = 1, linetype = "solid")) +
  labs(subtitle = paste('Res = ', res,', nPC = ',nPC, sep = ''), x = "UMAP 1", y = "UMAP 2") 


In [ ]:
DimPlot(pz.literature, reduction = "tsne", label = T) + 
  theme(plot.title = element_text(color="blue", size=26, face="bold.italic"),
        axis.text.x = element_text(angle = 90, face = "bold", color = 'dodgerblue4', size=22, hjust =1), 
        axis.title.x = element_text(face = "bold", color = "dodgerblue2", size = 24),
        axis.text.y = element_text(angle = 0, face = "bold", color = 'dodgerblue4', size=22),
        axis.title.y = element_text(face = "bold", color = "dodgerblue2", size = 24),
        legend.text = element_text(face = "bold", color = "dodgerblue2", size = 22),
        panel.background = element_rect(fill = "white",colour = "black", size = 1, linetype = "solid")) +
  labs(subtitle = paste('Res = ', res,', nPC = ',nPC, sep = ''), x = "UMAP 1", y = "UMAP 2") 

# resolution impact

In [ ]:
for (resolu in 1:10) {
  print(resolu/10)
  pz_tmp <- pz
  pz_tmp <- FindNeighbors(pz_tmp, dims = 1:nPC)
  pz_tmp <- FindClusters(pz_tmp, resolution = resolu/10) 
  RunUMAP(pz_tmp, dims = 1:nPC)
  assign(paste('plot_res', resolu/10, sep=''), DimPlot(pz_tmp, reduction = "umap", label = TRUE))
}

In [ ]:
options(repr.plot.width=17, repr.plot.height=17)
CombinePlots(plots = list(plot_res0.1, plot_res0.2, plot_res0.3, 
                          plot_res0.4, plot_res0.5, plot_res0.6, 
                          plot_res0.7, plot_res0.8, plot_res1))

# Feature plots 

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
FeaturePlot(pz, reduction = "umap", pt.size = 2, features = 'percent.mt', label = T)
FeaturePlot(pz, reduction = "umap", pt.size = 2, features = 'nFeature_RNA', label = T)

# Marker genes 

In [ ]:
thresh.use = 0.25
min.pct = 0.25
min.diff.pct = -Inf
test="wilcox"
test.use = test
cluster.markers = FindAllMarkers(pz, thresh.use = thresh.use, test.use=test.use, min.pct=min.pct, min.diff.pct=min.diff.pct, only.pos=TRUE)
filename_xls <- paste('pz',Npz,'_allmarkers_res',res,'_nPC',nPC,'min500cells.xlsx',sep='')

write.xlsx(cluster.markers,
           file= filename_xls, 
           row.names = T,
           asTable = T)

top10 <- cluster.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_logFC)
top3  <- cluster.markers %>% group_by(cluster) %>% top_n(n = 3, wt = avg_logFC)

DoHeatmap(pz, features = top10$gene, angle = 90) + NoLegend()


In [ ]:
options(repr.plot.width= 17, repr.plot.height=17)
DoHeatmap(pz, features = top10$gene, angle = 90) + NoLegend()

In [ ]:
#### Gene signature
signature = list()
SERTOLI_markers = c('AMH', 'SOX9', 'CLDN11', 'FATE1') #, 
#                    'RP13-49I15.5')
signature[['SERTOLI']] <- SERTOLI_markers
LEYDIG_markers = c('CALB2', 'INSL3', 'DLK1', 'LHCGR')#, 
#                   'CFD', 'IGFBP3', 'IGFBP5', 'IGF2', 'SLC25A37', 'GSTA1', 'SYCP3')
signature[['LEYDIG']] <- LEYDIG_markers
MYOID_markers = c('MYH11', 'ACTA2', 'MYL9', 'DES')
signature[['MYOID']] <- MYOID_markers
ENDOTHELIAL_markers = c('VWF', 'CD36', 'CD34', 'PECAM1')#, 
#                        'NOTCH4', 'JAG1', 'HES1', 'PALMD', 'PDGFB', 'TGFBR2', 'RGS5', 'EPAS1', 'NOSTRIN')
signature[['ENDOTHELIAL']] <- ENDOTHELIAL_markers
MACROPHAGE_markers = c('CD14', 'CD68', 'CD86', 'CD163')#, 
#                        'C1QA', 'HLA-DRB1', 'HLA-DPB1', 'CCR5', 'CD74', 'RGS1', 
#                        'CXCR4', 'TYROBP', 'CSF1R', 'MSR1', 'S100A4')
signature[['MACROPHAGE']] <- MACROPHAGE_markers


In [ ]:
cell_types <- c("SERTOLI", "LEYDIG" , "MYOID", "ENDOTHELIAL", "MACROPHAGE")
x<-pz

for (s in cell_types) {
  print(s)
  x[["Sign_exp"]] <- apply(FetchData(object = x, vars = signature[[s]]),1,mean)
  assign(paste('Plot_Sign',s,sep=''), 
         FeaturePlot(x, reduction = "umap", 
                     features = 'Sign_exp', 
                     label = T, 
                     cols = c("lightgrey", "red")) +
         theme(plot.title = element_text(color="blue", size=22, face="bold.italic"),
               plot.subtitle = element_text(color="dodgerblue2", size=16, face="italic"),
               axis.text.x = element_text(angle = 90, face = "bold", color = 'dodgerblue4', size=16, hjust =1), 
               axis.title.x = element_text(face = "bold", color = "dodgerblue2", size = 18),
               axis.text.y = element_text(angle = 0, face = "bold", color = 'dodgerblue4', size=16),
               axis.title.y = element_text(face = "bold", color = "dodgerblue2", size = 18),
               legend.text = element_text(face = "bold", color = "dodgerblue2", size = 12),
               panel.background = element_rect(fill = "white",colour = "black", size = 1, linetype = "solid")) +
         labs(title= 'Signture expression', subtitle = paste(s,' - MG: ',toString(signature[[s]]), sep=''), x = "UMAP 1", y = "UMAP 2")) 
}
options(repr.plot.width=12, repr.plot.height=10)
CombinePlots(plots = list(Plot_SignSERTOLI, 
             Plot_SignLEYDIG,
             Plot_SignMYOID,
             Plot_SignMACROPHAGE,
             Plot_SignENDOTHELIAL,
             Plot_SignLEYDIG_MYOID), ncol = 2)


In [ ]:
#iNOA1
new.cluster.ids <- c("low_quality",
                     "LEY",
                     "MYD", 
                     "SRT",
                     "MCR",
                     "END",
                     "TCL")
names(new.cluster.ids.lit) <- levels(pz)
pz <- RenameIdents(pz, new.cluster.ids.lit)

levels(pz)
pz@meta.data$cell_type=Idents(pz)
pz$cell_type <- pz@meta.data$celltype

cell2remove <- colnames(pz)[pz@meta.data$celltype == 'low_quality']
str(cell2remove)
options(repr.plot.width=17, repr.plot.height=7)
DimPlot(pz, cells.highlight = cell2remove, order = T, pt.size = 2)
pz.clean <- subset(pz, cells = cell2remove, invert=T)
save(pz.clean, "pz.iNOA1")

In [ ]:
#iNOA2
new.cluster.ids <- c("LEY",
                     "MYD",
                     "SRT",
                     "MCR",
                     "TCL")
names(new.cluster.ids.lit) <- levels(pz)
pz <- RenameIdents(pz, new.cluster.ids.lit)

levels(pz)
pz@meta.data$cell_type=Idents(pz)
pz$cell_type <- pz@meta.data$celltype
save(pz, "pz.iNOA2")

In [ ]:
#iNOA2
new.cluster.ids <- c("LEY",
                     "MYD",
                     "END",
                     "MCR",
                     "STRO",
                     "SRT",
                     "TCL")
names(new.cluster.ids.lit) <- levels(pz)
pz <- RenameIdents(pz, new.cluster.ids.lit)

levels(pz)
pz@meta.data$cell_type=Idents(pz)
pz$cell_type <- pz@meta.data$celltype
save(pz, "pz.iNOA3")

In [ ]:
#OA
new.cluster.ids <- c("MCR",
                     "Sperm",
                     "Sperm",
                     "Primary S'cytes",
                     "Differentiated S'gonia",                     
                     "Elong. S'tidis",
                     "Elong. S'tidis",
                     "Round S'tidis" ,                    
                     "SRT",
                     "LEY")
names(new.cluster.ids.lit) <- levels(pz)
pz <- RenameIdents(pz, new.cluster.ids.lit)

levels(pz)
pz@meta.data$cell_type=Idents(pz)
pz$cell_type <- pz@meta.data$celltype
save(pz, "pz.OA")

In [ ]:
# only for GSE112013
new.cluster.ids <- c("Elong. S'tidis",
                     "Sperm",
                     "Early primary S'cytes",
                     "Elong. S'tidis",
                     "LEY",
                     "END",
                     "MCR",
                     "MYD",
                     "SSCs",
                     "Round S'tidis",
                     "Late primary S'cyte",
                     "Differentiated S'gonia",
                     "SRT",
                     "STRO")
names(new.cluster.ids.lit) <- levels(pz)
pz <- RenameIdents(pz, new.cluster.ids.lit)

levels(pz)
pz@meta.data$cell_type=Idents(pz)
pz$cell_type <- pz@meta.data$celltype

pz.GSE112013 <- pz
save(pz.literature, file = 'pz.GSE112013')

In [ ]:
VlnPlot(pz, feature = c('VWF','PECAM1','NOTCH4', 'JAG1', 'HES1', 'MAML1'), slot = "counts", log = TRUE)

In [ ]:
VlnPlot(pz, feature = c('MYH11','ACTA2','PTCH1', 'PTCH2', 'GLI', 'IGFBP6'), slot = "counts", log = TRUE)

In [ ]:
VlnPlot(pz, feature = c('SOX9','FATE1','ITGA6', 'WFDC2', 'BEX2', 'PRND'), slot = "counts", log = TRUE)

In [ ]:
VlnPlot(pz, feature = c('CFD', 'LUM', 'C7'), slot = "counts", log = TRUE)

In [ ]:
VlnPlot(pz, feature = c('DLK1', 'INSL3', 'CFD', 'GSTA1', 'IGFBP3', 'CALB2'), slot = "counts", log = TRUE)

In [ ]:
VlnPlot(pz, feature = c('MAGEA4', 'TNP1', 'ZPBP','PRM2','DAZL', 'ID4','FGFR3','KIT','DDX4'), slot = "counts", log = TRUE)

In [ ]:
DimPlot(pz, reduction = "umap", label = T, pt.size = 2, label.size = 7) + 
  theme(plot.title = element_text(color="blue", size=26, face="bold.italic"),
        axis.text.x = element_text(angle = 90, face = "bold", color = 'dodgerblue4', size=22, hjust =1), 
        axis.title.x = element_text(face = "bold", color = "dodgerblue2", size = 24),
        axis.text.y = element_text(angle = 0, face = "bold", color = 'dodgerblue4', size=22),
        axis.title.y = element_text(face = "bold", color = "dodgerblue2", size = 24),
        legend.text = element_text(face = "bold", color = "dodgerblue2", size = 22),
        panel.background = element_rect(fill = "white",colour = "black", size = 1, linetype = "solid")) +
  labs(subtitle = paste('pz_',Npz,'- Res = ',res,', nPC =', nPC, sep = ' '), x = "UMAP 1", y = "UMAP 2") 

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
DoHeatmap(pz, 
          features = top10$gene, 
          angle = 90) + NoLegend()

In [ ]:
load('pz.GSE112013')
load('pz.OA')
testis.query <- pz.OA
testis.anchors <- FindTransferAnchors(reference = pz.GSE112013, query = testis.query, 
    dims = 1:30)
predictions <- TransferData(anchorset = testis.anchors, refdata = pz.literature$cell_type, 
    dims = 1:30)
testis.query <- AddMetaData(testis.query, metadata = predictions)
table(testis.query$predicted.id)
testis.query$prediction.match <- testis.query$predicted.id == testis.query$cell_type
table(testis.query$prediction.match)